# Error mitigation in measurement

we have seen in the introduction that different type of errors could happen at different times of the execution. In the previous posts we focused on fixing the storage errors (when we storage a qubit state for some time...) but what about the errors at the time of meassurement?. This is a little more complicated matter because we can't use the same tricks because we the error is located elsewhere. So how to address this problem? Well, we'll use statistics to correct this type of error in the following way:

First, let's import some functions that allow us to convert bit strings to strings and viceversa. Then let's define a function that codificates the received bit string into quanutm bits. This codification is really simple because we only represent a bit using quantum its. We won't use super postion to this excercise in part because we are only interested in meassuring errors. Finally, the last function count the number of 0 and 1 from an array of bits.

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, ClassicalRegister
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import lookuptable_decoding
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)
from qiskit.visualization import plot_histogram
backend = Aer.get_backend('qasm_simulator')

def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def get_noise(p):

    error_meas = pauli_error([('X',p), ('I', 1 - p)])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") 
        
    return noise_model

def codificate(bitString):
    qubits = list()
    for i in range(len(bitString)):
        mycircuit = QuantumCircuit(1,1)
        if(bitString[i] == "1"):
            mycircuit.x(0)
        qubits.append(mycircuit)
    return qubits

def count(array):
    numZero = 0
    numOne = 0
    for i in array:
        if i == "0":
            numZero += 1 
        elif i == "1":
            numOne += 1
    return dict({"0":numZero, "1":numOne})

As we saw in the previous posts, let's see the result without any errors:

In [17]:
m0 = tobits("I like dogs") 
qubits = codificate(m0)
measurements = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print(frombits(measurements))

I like dogs


Let's introduce some errors when meassuring and see the results. It's unreadeable as expected.

In [18]:
m0 = tobits("I like dogs") 
qubits = codificate(m0)
measurements = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True, noise_model=get_noise(0.2)).result()
    measurements.append(int(result.get_memory()[0]))
print(frombits(measurements))

fl`ked$ÿW{


Let's see how the error correction helps in this situation. 

First we'll run a meassurement 2000 times. This measurement is especial because we know that without noise we have to meassure a "0" 1000 times and then read "1" 1000 times. Then, let's count the outputs to construct the matrix M. This matrix has some statistics of when we got a 0 and when we got a 1, given the fact that we already know what was the expected output.

In [4]:
for state in ['0','1']:
    qc = QuantumCircuit(1,1)
    if state[0]=='1':
        qc.x(0)  
    qc.measure(qc.qregs[0],qc.cregs[0])
    print(state+' becomes',
          execute(qc, Aer.get_backend('qasm_simulator'),noise_model=get_noise(0.2),shots=1000).result().get_counts())

0 becomes {'0': 389, '1': 611}
1 becomes {'0': 593, '1': 407}


Then, let's make a meassurement to get the $C_{noisy}$. This matrix has the counts of the result of doing a meassurement of the i-th bit of our original bit string.

In [5]:
qubit = qubits[0]
result = execute(qubit, backend, shots=1000, memory=True, noise_model=get_noise(0.2)).result()
print(result.get_counts())

{'0': 397, '1': 603}


Then we apply the following operation to get the error mitigation. $C_{noisy} = M ~ C_{ideal} \rightarrow C_{ideal} = M^{-1} C_{noisy}.$

In [3]:
import numpy as np
import scipy.linalg as la

M = [[0.389,0.611],
    [0.593,0.407]]
Cnoisy = [[397],[603]]

Minv = la.inv(M)

Cmitigated = np.dot(Minv, Cnoisy)
print('C_mitigated =\n',Cmitigated)

C_mitigated =
 [[1013.99019608]
 [   4.18627451]]


If we take only the positive part and we take the floor of the numbers (there aren't half meassurements or negative meassurements) then we have the error correction for this qubit only. We need to repeat the process for every qubit....

In [9]:
Cmitigated = np.floor(Cmitigated)
if(Cmitigated[1][0] < 0):
    Cmitigated[1][0] = 0
print('C_mitigated =\n',Cmitigated)

C_mitigated =
 [[1013.]
 [   4.]]


Insted of doing this process by hand we can use the qiskit libraries to build the matrices.

In [13]:
qr = QuantumRegister(1)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
backend = Aer.get_backend('qasm_simulator')
job = execute(meas_calibs, backend=backend, shots=1000,noise_model=get_noise(0.2))
cal_results = job.result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
print(meas_fitter.cal_matrix)

[[0.807 0.195]
 [0.193 0.805]]


As you can see we have created the M matrix without all of the hardwork. Then we can create the $C_{noisy}$ matrix in the same way as before.

In [14]:
qubit = qubits[2]
qubit.measure(0,0)
results = execute(qubit, backend=backend, shots=10000, noise_model=get_noise(0.2),memory=True).result()
noisy_counts = count(results.get_memory())
print(noisy_counts)

{'0': 5398, '1': 4602}


And then we can calculate the result in the same way.

In [15]:
Minv = la.inv(meas_fitter.cal_matrix)

Cmitigated = np.dot(Minv, np.array(list(noisy_counts.values())))
print('C_mitigated =\n',Cmitigated)

C_mitigated =
 [5633.9869281 4366.0130719]


Finally if we apply this process to all qubits, we'll have as a result:

In [19]:
outputs = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,0)
    results = execute(qubit, backend=backend, shots=10000, memory=True, noise_model=get_noise(0.2)).result()
    noisy_counts = count(results.get_memory())
    Minv = la.inv(meas_fitter.cal_matrix)
    Cmitigated = np.dot(Minv, np.array(list(noisy_counts.values())))
    if(Cmitigated[0] > Cmitigated[1]):
        outputs.append('0')
    else:
        outputs.append('1')
print(frombits(''.join(outputs)))

I like dogs


But what if we didn't want to make operations with linear algebra?
Well, qiskit has a built in function that do all these operations for us

In [21]:
outputs = list()
meas_filter = meas_fitter.filter
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,0)
    results = execute(qubit, backend=backend, shots=10000, memory=True, noise_model=get_noise(0.2)).result()
    noisy_counts = count(results.get_memory())
    # Results with mitigation
    mitigated_counts = meas_filter.apply(noisy_counts)
    if('1' not in mitigated_counts):
        outputs.append('0')
    elif('0' not in mitigated_counts):
        outputs.append('1')
    elif(mitigated_counts['0'] > mitigated_counts['1']):
        outputs.append('0')
    else:
        outputs.append('1')
print(frombits(''.join(outputs)))

I like dogs


As you can see is a lot easier to create the error mitigation in this way. We avoid doing by hand everything.

Something that you may have notice is that even if we had the correct string in the end in the $C_{mitigated}$ matrices we have counts that were really low but they were there and they weren't our expected result. This is why this is a mitigation algorithm. Unlike the codes from the previous post where we could correct the error completly, in the mitigation algorithms we'll have some noise even after the algorithm is applied.

## References:
https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html